In [3]:
import sqlite3
import pickle
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Queue


### Dump train data (continuous)

In [4]:
conn = sqlite3.connect('data.dat')
cursor = conn.cursor()

cursor.execute("SELECT IMAGE, RESPONSE FROM TRAIN")
result = cursor.fetchall()

In [5]:
image = [pickle.loads(r[0]).reshape(-1) for r in result]

In [6]:
response = [r[1] for r in result]

In [8]:
pickle.dump(image, open('image.dat', 'wb'))
pickle.dump(response, open('response.dat', 'wb'))

In [5]:
clf = SVR(C=100, epsilon=0.05, gamma='auto', kernel='rbf', shrinking=True)

In [6]:
clf.fit(image, response)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.05, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [11]:
clf.predict(image[0:3])

array([-0.25044157, -0.25044157, -0.55044157])

In [9]:
response[10]

-0.9

### Splite dataset

In [33]:
clf = SVR(kernel='rbf', shrinking=True, epsilon=0.05, )
clf.fit(image, response)
cost(clf.predict(image), response)
pickle.dump(clf, open('svr_predict', 'wb'))

In [6]:
x_train, x_test, y_train, y_test = train_test_split(image, response,
                                                   test_size=0.2, random_state=0)

In [45]:
clf2 = pickle.load(open('svr_predict', 'rb'))
print(clf2.predict(image[100]))
print(response[100])

[ 0.95017462]
1.0


F:\Applications\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


### Choose c and gamma

In [8]:
range_c = (1, 1000)
range_gamma = (1/494, 10)
iter_round = 10
cs = []
gammas = []
variances = []
biases = []
data_queue = Queue()

def generate_parameter():
    for i in range(0, iter_round):
        c = (range_c[1] - range_c[0])/iter_round + range_c[0]
        for j in range(0, iter_round):
            g = (range_gamma[1] - range_gamma[0])/iter_round + range_gamma[0]
            yield c, g

# vectors
def cost(x, y):
    x_array = np.array(x)
    y_array = np.array(y)
    square = x_array.dot(y_array)
    return square/y_array.shape[0]

def calcu(c, gamma, x_tr, y_tr, x_te, y_te, queue):
    clf = SVR(C=c, epsilon=0.05, kernel='rbf', gamma=gamma, shrinking=True)
    clf.fit(x_tr, y_tr)
    var = cost(clf.predict(x_tr), y_tr)
    bias = cost(clf.predict(x_te), y_te)
    queue.put((c, gamma, var, bias))
    print('c: %f', 'gamma: %f' % (c,gamma))

"""
with ProcessPoolExecutor(max_workers=4) as executor:
    for c, gamma in generate_parameter():
        executor.submit(calcu, c, gamma, x_train, y_train, x_test, y_test,
                       data_queue)
"""


'\nwith ProcessPoolExecutor(max_workers=4) as executor:\n    for c, gamma in generate_parameter():\n        executor.submit(calcu, c, gamma, x_train, y_train, x_test, y_test,\n                       data_queue)\n'

### No parallel

In [21]:
range_c = (1, 1000)
range_gamma = (1/494, 10)
iter_round = 10
cs = []
gammas = []
variances = []
biases = []
for c, gamma in generate_parameter():
    clf = SVR(C=c, epsilon=0.05, kernel='rbf', gamma=gamma, shrinking=True)
    clf.fit(x_train, y_train)
    var = cost(clf.predict(x_train), y_train)
    bias = cost(clf.predict(x_test), y_test)
    cs.append(c)
    gammas.append(gamma)
    variances.append(var)
    biases.append(bias)
    print(c, gamma, var, bias)

pickle.dump(zip(cs, gammas, variances, biases), open('para_1_1000_1/494_10','wb'))


1.0 0.0020242914979757085 0.371549472028 0.123523464921
1.0 1.001821862348178 0.371622527575 0.123479600049
1.0 2.0016194331983805 0.371622527575 0.123479600049
1.0 3.001417004048583 0.371622527575 0.123479600049
1.0 4.0012145748987855 0.371622527575 0.123479600049
1.0 5.001012145748987 0.371622527575 0.123479600049
1.0 6.00080971659919 0.371622527575 0.123479600049
1.0 7.000607287449392 0.371622527575 0.123479600049
1.0 8.000404858299595 0.371622527575 0.123479600049
1.0 9.000202429149798 0.371622527575 0.123479600049
100.9 0.0020242914979757085 0.376292984679 0.124094439754
100.9 1.001821862348178 0.376270285244 0.124041642359
100.9 2.0016194331983805 0.376270285244 0.124041642359
100.9 3.001417004048583 0.376270285244 0.124041642359
100.9 4.0012145748987855 0.376270285244 0.124041642359
100.9 5.001012145748987 0.376270285244 0.124041642359
100.9 6.00080971659919 0.376270285244 0.124041642359
100.9 7.000607287449392 0.376270285244 0.124041642359
100.9 8.000404858299595 0.376270285244

FileNotFoundError: [Errno 2] No such file or directory: 'para_1_1000_1/494_10'

### Plot 